In [1]:
# Importing Libraries
import numpy as np
import scipy as sp
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import AdaBoostRegressor, AdaBoostClassifier, \
    GradientBoostingClassifier, GradientBoostingRegressor, \
    RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import r2_score
from pymatgen.core import Element, Composition, periodic_table
from functools import partial


In [2]:
#  --NotebookApp.iopub_data_rate_limit=1.0e10


In [6]:
# Loading training and testing data
train= pd.read_csv("train.csv",index_col=False)
test= pd.read_csv("test.csv",index_col=False)

# Querying materials and their properties based on training data
from pymatgen.ext.matproj import MPRester
mpr = MPRester("241iWwhTEOaNmC6V")

# Using material IDs provided in training data to get corresponding information from MPD
data = mpr.query(criteria={"task_id": {"$in":train["material_id"].to_list()}}, properties=["material_id","energy",
        "energy_per_atom",
        "volume",
        "formation_energy_per_atom",
        "nsites",
        "pretty_formula",                                                                                  
        "nelements",
        "density",  "band_gap"])
df = pd.DataFrame(data)
display(df)

  0%|          | 0/5619 [00:00<?, ?it/s]

,material_id,energy,energy_per_atom,volume,formation_energy_per_atom,nsites,pretty_formula,nelements,density,band_gap
0,mp-1001034,-52.019078,-3.715648,376.145863,-0.703661,14,Mg(InSe2)2,3,5.030727,0.7432
1,mp-1001780,-22.364406,-5.591101,78.515319,-1.699567,4,LuCuS2,3,6.400668,1.5031
2,mp-1001786,-23.600913,-5.900228,71.701237,-2.103385,4,LiScS2,3,2.687084,1.5296
3,mp-1002124,-19.780759,-9.890380,32.055765,-0.298402,2,HfC,2,9.868236,0.5774
4,mp-1004528,-95.454964,-5.614998,324.979430,-3.162482,17,CsB3PbF12,4,3.068458,6.3125
...,...,...,...,...,...,...,...,...,...,...
5614,mvc-6916,-99.385289,-7.098949,156.686110,-2.307339,14,MgMn2O4,3,4.200544,0.0516
5615,mvc-6928,-91.603914,-6.543137,173.941832,-2.192463,14,Ca(FeO2)2,3,4.119621,1.7463
5616,mvc-6946,-36.735653,-6.122609,97.529160,-1.961400,6,SnO2,2,5.131960,2.1009
5617,mvc-7040,-145.212422,-8.067357,418.920727,-1.999722,18,Re2O7,2,3.840256,3.4689


In [7]:
# Making list of possible noble gases 
nobles = ["He","Ne", "Ar", "Kr", "Xe", "Rn", "Og"]
filt_data = df

#Iterating through data and using pandas drop function to filter out materials containing noble gas elements 
for i in nobles:
    filt_data = filt_data.drop(filt_data[filt_data['pretty_formula'].str.contains(i)].index)
    
n = len(filt_data)
print(f'There are {n} materials left')    
display(filt_data)


print(f'There are {n} materials left')    
display(filt_data)


There are 5614 materials left


,material_id,energy,energy_per_atom,volume,formation_energy_per_atom,nsites,pretty_formula,nelements,density,band_gap
0,mp-1001034,-52.019078,-3.715648,376.145863,-0.703661,14,Mg(InSe2)2,3,5.030727,0.7432
1,mp-1001780,-22.364406,-5.591101,78.515319,-1.699567,4,LuCuS2,3,6.400668,1.5031
2,mp-1001786,-23.600913,-5.900228,71.701237,-2.103385,4,LiScS2,3,2.687084,1.5296
3,mp-1002124,-19.780759,-9.890380,32.055765,-0.298402,2,HfC,2,9.868236,0.5774
4,mp-1004528,-95.454964,-5.614998,324.979430,-3.162482,17,CsB3PbF12,4,3.068458,6.3125
...,...,...,...,...,...,...,...,...,...,...
5614,mvc-6916,-99.385289,-7.098949,156.686110,-2.307339,14,MgMn2O4,3,4.200544,0.0516
5615,mvc-6928,-91.603914,-6.543137,173.941832,-2.192463,14,Ca(FeO2)2,3,4.119621,1.7463
5616,mvc-6946,-36.735653,-6.122609,97.529160,-1.961400,6,SnO2,2,5.131960,2.1009
5617,mvc-7040,-145.212422,-8.067357,418.920727,-1.999722,18,Re2O7,2,3.840256,3.4689


There are 5614 materials left


,material_id,energy,energy_per_atom,volume,formation_energy_per_atom,nsites,pretty_formula,nelements,density,band_gap
0,mp-1001034,-52.019078,-3.715648,376.145863,-0.703661,14,Mg(InSe2)2,3,5.030727,0.7432
1,mp-1001780,-22.364406,-5.591101,78.515319,-1.699567,4,LuCuS2,3,6.400668,1.5031
2,mp-1001786,-23.600913,-5.900228,71.701237,-2.103385,4,LiScS2,3,2.687084,1.5296
3,mp-1002124,-19.780759,-9.890380,32.055765,-0.298402,2,HfC,2,9.868236,0.5774
4,mp-1004528,-95.454964,-5.614998,324.979430,-3.162482,17,CsB3PbF12,4,3.068458,6.3125
...,...,...,...,...,...,...,...,...,...,...
5614,mvc-6916,-99.385289,-7.098949,156.686110,-2.307339,14,MgMn2O4,3,4.200544,0.0516
5615,mvc-6928,-91.603914,-6.543137,173.941832,-2.192463,14,Ca(FeO2)2,3,4.119621,1.7463
5616,mvc-6946,-36.735653,-6.122609,97.529160,-1.961400,6,SnO2,2,5.131960,2.1009
5617,mvc-7040,-145.212422,-8.067357,418.920727,-1.999722,18,Re2O7,2,3.840256,3.4689


In [8]:
listA = [a for a in filt_data["pretty_formula"]]
listB = [Composition(a) for a in listA]
listC = [a.elements for a in listB]

editC = [item for sublist in listC for item in sublist]
unique_editC = set(editC)
unique_editC

UL_editC = [a for a in unique_editC]
cprops = [a.data for a in UL_editC]


In [9]:
peel=set(UL_editC)
sy_editC = [a.symbol for a in UL_editC]
sy_editC


pd.set_option('display.max_columns', None)

cprops_df = pd.DataFrame(cprops, index=sy_editC)

cprops_df.head()

,Atomic mass,Atomic no,Atomic orbitals,Atomic radius,Atomic radius calculated,Boiling point,Brinell hardness,Bulk modulus,Coefficient of linear thermal expansion,Common oxidation states,Critical temperature,Density of solid,Electrical resistivity,Electronic structure,ICSD oxidation states,Liquid range,Melting point,Mendeleev no,Mineral hardness,Molar volume,Name,Oxidation states,Poissons ratio,Reflectivity,Refractive index,Rigidity modulus,Shannon radii,Superconduction temperature,Thermal conductivity,Van der waals radius,Velocity of sound,Vickers hardness,X,Youngs modulus,NMR Quadrupole Moment,Metallic radius,iupac_ordering,IUPAC ordering,Ground level,Ionization energies,Electron affinity,Ionic radii,Ionic radii hs,Ionic radii ls
H,1.007940,1,{'1s': -0.233471},0.25,0.53,20.28 K,no data MN m<sup>-2</sup>,no data GPa,no data x10<sup>-6</sup>K<sup>-1</sup>,"[-1, 1]",33 K,no data kg m<sup>-3</sup>,no data 10<sup>-8</sup> &Omega; m,1s<sup>1</sup>,"[1, -1]",6.27 K,14.01 K,103,no data,11.42 cm<sup>3</sup>,Hydrogen,"[-1, 1]",no data,no data %,1.000132 (gas; liquid 1.12)(no units),no data GPa,"{'1': {'I': {'': {'crystal_radius': -0.24, 'io...",no data K,0.1805 W m<sup>-1</sup> K<sup>-1</sup>,1.10,1270 m s<sup>-1</sup>,no data MN m<sup>-2</sup>,2.20,no data GPa,{'H-2': 2.86},no data,92,92,2S1/2,[13.598434599702],0.754598,NaN,NaN,NaN
Li,6.941000,3,"{'1s': -1.878564, '2s': -0.10554}",1.45,1.67,1615 K,no data MN m<sup>-2</sup>,11 GPa,46 x10<sup>-6</sup>K<sup>-1</sup>,[1],3223 K,535 kg m<sup>-3</sup>,9.5 10<sup>-8</sup> &Omega; m,[He].2s<sup>1</sup>,[1],1161.31 K,453.69 K,12,0.6,13.02 cm<sup>3</sup>,Lithium,[1],no data,no data %,no data,4.2 GPa,"{'1': {'IV': {'': {'crystal_radius': 0.73, 'io...",no data K,85 W m<sup>-1</sup> K<sup>-1</sup>,1.82,6000 m s<sup>-1</sup>,no data MN m<sup>-2</sup>,0.98,4.9 GPa,"{'Li-6': -0.808, 'Li-7': -40.1}",1.52,11,11,2S1/2,"[5.391714996, 75.640097, 122.45435914]",0.618049,{'1': 0.9},NaN,NaN
Be,9.012182,4,"{'1s': -3.856411, '2s': -0.205744}",1.05,1.12,2742 K,600 MN m<sup>-2</sup>,130 GPa,11.3 x10<sup>-6</sup>K<sup>-1</sup>,[2],no data K,1848 kg m<sup>-3</sup>,3.8 10<sup>-8</sup> &Omega; m,[He].2s<sup>2</sup>,[2],1182 K,1560 K,77,5.5,4.85 cm<sup>3</sup>,Beryllium,[2],0.032,no data %,no data,132 GPa,"{'2': {'III': {'': {'crystal_radius': 0.3, 'io...",0.026 K,190 W m<sup>-1</sup> K<sup>-1</sup>,1.53,13000 m s<sup>-1</sup>,1670 MN m<sup>-2</sup>,1.57,287 GPa,{'Be-9': 52.88},1.12,17,17,1S0,"[9.322699, 18.21115, 153.896205, 217.7185861]",-0.520000,{'2': 0.59},NaN,NaN
B,10.811000,5,"{'1s': -6.564347, '2p': -0.136603, '2s': -0.34...",0.85,0.87,4200 K,no data MN m<sup>-2</sup>,320 GPa,6 x10<sup>-6</sup>K<sup>-1</sup>,[3],no data K,2460 kg m<sup>-3</sup>,&gt; 10<sup>12</sup>10<sup>-8</sup> &Omega; m,[He].2s<sup>2</sup>.2p<sup>1</sup>,"[3, -3]",1851 K,2349 K,86,9.3,4.39 cm<sup>3</sup>,Boron,"[1, 2, 3]",no data,no data %,no data,no data GPa,"{'3': {'III': {'': {'crystal_radius': 0.15, 'i...",no data K,27 W m<sup>-1</sup> K<sup>-1</sup>,1.92,16200 m s<sup>-1</sup>,49000 MN m<sup>-2</sup>,2.04,no data GPa,"{'B-10': 84.59, 'B-11': 40.59}",no data,81,81,2P°1/2,"[8.298019, 25.15483, 37.93059, 259.3715, 340.2...",0.279723,{'3': 0.41},NaN,NaN
C,12.010700,6,"{'1s': -9.947718, '2p': -0.199186, '2s': -0.50...",0.70,0.67,4300 K,no data MN m<sup>-2</sup>,33 GPa,7.1 x10<sup>-6</sup>K<sup>-1</sup>,"[-4, 4]",no data K,2267 kg m<sup>-3</sup>,about 1000 - direction dependent10<sup>-8</sup...,[He].2s<sup>2</sup>.2p<sup>2</sup>,"[2, 3, 4, -4, -3, -2]",500 K,3800 K,95,0.5 (graphite; diamond is 10.0)(no units),5.29 cm<sup>3</sup>,Carbon,"[-4, -3, -2, -1, 1, 2, 3, 4]",no data,27 %,2.417 (diamond)(no units),no data GPa,"{'4': {'III': {'': {'crystal_radius': 0.06, 'i...",no data K,140 W m<sup>-1</sup> K<sup>-1</sup>,1.70,18350 m s<sup>-1</sup>,no data MN m<sup>-2</sup>,2.55,no data GPa,{'C-11': 33.27},no data,86,86,3P0,"[11.260288, 24.383154, 47.88778, 64.49352, 392...",1.262114,{'4': 0.3},NaN,NaN


In [10]:
droplist = ['Ionic radii',
    'Ionic radii hs', 
     'Ionic radii ls',
     'iupac_ordering', 
     'IUPAC ordering', 
     'NMR Quadrupole Moment', 
     'Reflectivity',
     'Refractive index', 
     'Rigidity modulus', 
     'Shannon radii',
     'Superconduction temperature',
     'Mendeleev no',       
     'Mineral hardness',
     'Molar volume',
     'Name',
     'Oxidation states',
     'ICSD oxidation states',
     'Brinell hardness',
     'Atomic orbitals', 
     'Coefficient of linear thermal expansion',
     'Atomic orbitals',
     'Electronic structure',
     'Electrical resistivity',
     'Ground level'
           ]


cprops_df = cprops_df.drop(columns=droplist)
cprops_df

,Atomic mass,Atomic no,Atomic radius,Atomic radius calculated,Boiling point,Bulk modulus,Common oxidation states,Critical temperature,Density of solid,Liquid range,Melting point,Poissons ratio,Thermal conductivity,Van der waals radius,Velocity of sound,Vickers hardness,X,Youngs modulus,Metallic radius,Ionization energies,Electron affinity
H,1.007940,1,0.25,0.53,20.28 K,no data GPa,"[-1, 1]",33 K,no data kg m<sup>-3</sup>,6.27 K,14.01 K,no data,0.1805 W m<sup>-1</sup> K<sup>-1</sup>,1.10,1270 m s<sup>-1</sup>,no data MN m<sup>-2</sup>,2.20,no data GPa,no data,[13.598434599702],0.754598
Li,6.941000,3,1.45,1.67,1615 K,11 GPa,[1],3223 K,535 kg m<sup>-3</sup>,1161.31 K,453.69 K,no data,85 W m<sup>-1</sup> K<sup>-1</sup>,1.82,6000 m s<sup>-1</sup>,no data MN m<sup>-2</sup>,0.98,4.9 GPa,1.52,"[5.391714996, 75.640097, 122.45435914]",0.618049
Be,9.012182,4,1.05,1.12,2742 K,130 GPa,[2],no data K,1848 kg m<sup>-3</sup>,1182 K,1560 K,0.032,190 W m<sup>-1</sup> K<sup>-1</sup>,1.53,13000 m s<sup>-1</sup>,1670 MN m<sup>-2</sup>,1.57,287 GPa,1.12,"[9.322699, 18.21115, 153.896205, 217.7185861]",-0.520000
B,10.811000,5,0.85,0.87,4200 K,320 GPa,[3],no data K,2460 kg m<sup>-3</sup>,1851 K,2349 K,no data,27 W m<sup>-1</sup> K<sup>-1</sup>,1.92,16200 m s<sup>-1</sup>,49000 MN m<sup>-2</sup>,2.04,no data GPa,no data,"[8.298019, 25.15483, 37.93059, 259.3715, 340.2...",0.279723
C,12.010700,6,0.70,0.67,4300 K,33 GPa,"[-4, 4]",no data K,2267 kg m<sup>-3</sup>,500 K,3800 K,no data,140 W m<sup>-1</sup> K<sup>-1</sup>,1.70,18350 m s<sup>-1</sup>,no data MN m<sup>-2</sup>,2.55,no data GPa,no data,"[11.260288, 24.383154, 47.88778, 64.49352, 392...",1.262114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pb,207.200000,82,1.80,1.54,2022 K,46 GPa,"[2, 4]",no data K,11340 kg m<sup>-3</sup>,1421.39 K,600.61 K,0.44,35 W m<sup>-1</sup> K<sup>-1</sup>,2.02,1260 m s<sup>-1</sup>,no data MN m<sup>-2</sup>,2.33,16 GPa,1.75,"[7.4166799, 15.032499, 31.9373, 42.33256, 68.8...",0.356721
Bi,208.980400,83,1.60,1.43,1837 K,31 GPa,[3],no data K,9780 kg m<sup>-3</sup>,1292.6 K,544.4 K,0.33,8 W m<sup>-1</sup> K<sup>-1</sup>,2.07,1790 m s<sup>-1</sup>,no data MN m<sup>-2</sup>,2.02,32 GPa,1.82,"[7.285516, 16.703, 25.57075, 45.37, 54.856, 88...",0.942362
Ac,227.000000,89,1.95,no data,3573 K,no data GPa,[3],no data K,10070 kg m<sup>-3</sup>,2250 K,1323 K,no data,12 W m<sup>-1</sup> K<sup>-1</sup>,2.47,no data m s<sup>-1</sup>,no data MN m<sup>-2</sup>,1.10,no data GPa,1.878,"[5.380226, 11.75, 17.431, 44.8, 55.0, 67.0, 79...",0.350000
Th,232.038060,90,1.80,no data,5093 K,54 GPa,[4],no data K,11724 kg m<sup>-3</sup>,2978 K,2115 K,0.27,54 W m<sup>-1</sup> K<sup>-1</sup>,2.45,2490 m s<sup>-1</sup>,350 MN m<sup>-2</sup>,1.30,79 GPa,1.798,"[6.3067, 12.1, 18.32, 28.648, 58.0, 69.1, 82.0...",1.170000


In [11]:
#pd.set_option('display.max_rows', None)
cprops_df['Melting point']

H      14.01 K
Li    453.69 K
Be      1560 K
B       2349 K
C       3800 K
        ...   
Pb    600.61 K
Bi     544.4 K
Ac      1323 K
Th      2115 K
U     1405.3 K
Name: Melting point, Length: 80, dtype: object

In [12]:


cprops_df['Boiling point'] = [(a.replace('K', '', 1)) for a in cprops_df['Boiling point']]
cprops_df['Bulk modulus'] = cprops_df["Bulk modulus"].str.replace("GPa", "")
cprops_df['Critical temperature'] = cprops_df["Critical temperature"].str.replace("K", "")
cprops_df['Density of solid'] = cprops_df['Density of solid'].str.replace("no data", "NaN")
cprops_df['Density of solid'] = cprops_df["Density of solid"].str.replace("kg m<sup>-3</sup>", "")
cprops_df['Liquid range'] = [(a.replace('K', '', 1)) for a in cprops_df['Liquid range']]
cprops_df['Poissons ratio'] = [(a.replace('no data', 'NaN', 1)) for a in cprops_df['Poissons ratio']]
cprops_df['Poissons ratio'] = cprops_df['Poissons ratio'].str.replace("no data", "").astype(float)
cprops_df['Thermal conductivity'] = [float(a.replace('W m<sup>-1</sup> K<sup>-1</sup>', "", 1)) for a in cprops_df['Thermal conductivity']]
cprops_df['Velocity of sound'] = cprops_df['Velocity of sound'].str.replace("no data","NaN")
cprops_df['Velocity of sound'] = [(a.replace('m s<sup>-1</sup>', '', 1)) for a in cprops_df['Velocity of sound']]
cprops_df['Vickers hardness'] = cprops_df['Vickers hardness'].str.replace("no data", "NaN")
cprops_df['Vickers hardness'] = [(a.replace('MN m<sup>-2</sup>', '', 1)) for a in cprops_df['Vickers hardness']]
cprops_df['Youngs modulus'] = cprops_df['Youngs modulus'].str.replace("no data", "NaN")
cprops_df['Youngs modulus'] = [(a.replace('GPa', '', 1)) for a in cprops_df['Youngs modulus']]



#cprops_df['Boiling point'] = [float(a.replace('K', '', 1)) for a in cprops_df['Boiling point']]
cprops_df

,Atomic mass,Atomic no,Atomic radius,Atomic radius calculated,Boiling point,Bulk modulus,Common oxidation states,Critical temperature,Density of solid,Liquid range,Melting point,Poissons ratio,Thermal conductivity,Van der waals radius,Velocity of sound,Vickers hardness,X,Youngs modulus,Metallic radius,Ionization energies,Electron affinity
H,1.007940,1,0.25,0.53,20.28,no data,"[-1, 1]",33,NaN,6.27,14.01 K,NaN,0.1805,1.10,1270,NaN,2.20,NaN,no data,[13.598434599702],0.754598
Li,6.941000,3,1.45,1.67,1615,11,[1],3223,535,1161.31,453.69 K,NaN,85.0000,1.82,6000,NaN,0.98,4.9,1.52,"[5.391714996, 75.640097, 122.45435914]",0.618049
Be,9.012182,4,1.05,1.12,2742,130,[2],no data,1848,1182,1560 K,0.032,190.0000,1.53,13000,1670,1.57,287,1.12,"[9.322699, 18.21115, 153.896205, 217.7185861]",-0.520000
B,10.811000,5,0.85,0.87,4200,320,[3],no data,2460,1851,2349 K,NaN,27.0000,1.92,16200,49000,2.04,NaN,no data,"[8.298019, 25.15483, 37.93059, 259.3715, 340.2...",0.279723
C,12.010700,6,0.70,0.67,4300,33,"[-4, 4]",no data,2267,500,3800 K,NaN,140.0000,1.70,18350,NaN,2.55,NaN,no data,"[11.260288, 24.383154, 47.88778, 64.49352, 392...",1.262114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pb,207.200000,82,1.80,1.54,2022,46,"[2, 4]",no data,11340,1421.39,600.61 K,0.440,35.0000,2.02,1260,NaN,2.33,16,1.75,"[7.4166799, 15.032499, 31.9373, 42.33256, 68.8...",0.356721
Bi,208.980400,83,1.60,1.43,1837,31,[3],no data,9780,1292.6,544.4 K,0.330,8.0000,2.07,1790,NaN,2.02,32,1.82,"[7.285516, 16.703, 25.57075, 45.37, 54.856, 88...",0.942362
Ac,227.000000,89,1.95,no data,3573,no data,[3],no data,10070,2250,1323 K,NaN,12.0000,2.47,NaN,NaN,1.10,NaN,1.878,"[5.380226, 11.75, 17.431, 44.8, 55.0, 67.0, 79...",0.350000
Th,232.038060,90,1.80,no data,5093,54,[4],no data,11724,2978,2115 K,0.270,54.0000,2.45,2490,350,1.30,79,1.798,"[6.3067, 12.1, 18.32, 28.648, 58.0, 69.1, 82.0...",1.170000


In [13]:
cprops_df['Bulk modulus'] = [(a.replace('no data', 'NaN', 1)) for a in cprops_df['Bulk modulus']]
cprops_df['Bulk modulus'] = [(a.replace('liquid', '', 1)) for a in cprops_df['Bulk modulus']]
cprops_df['Bulk modulus'] = cprops_df['Bulk modulus'].str.replace(r"\(.*\)","",  regex=True).astype(float)
cprops_df 
#cprops_df


,Atomic mass,Atomic no,Atomic radius,Atomic radius calculated,Boiling point,Bulk modulus,Common oxidation states,Critical temperature,Density of solid,Liquid range,Melting point,Poissons ratio,Thermal conductivity,Van der waals radius,Velocity of sound,Vickers hardness,X,Youngs modulus,Metallic radius,Ionization energies,Electron affinity
H,1.007940,1,0.25,0.53,20.28,NaN,"[-1, 1]",33,NaN,6.27,14.01 K,NaN,0.1805,1.10,1270,NaN,2.20,NaN,no data,[13.598434599702],0.754598
Li,6.941000,3,1.45,1.67,1615,11.0,[1],3223,535,1161.31,453.69 K,NaN,85.0000,1.82,6000,NaN,0.98,4.9,1.52,"[5.391714996, 75.640097, 122.45435914]",0.618049
Be,9.012182,4,1.05,1.12,2742,130.0,[2],no data,1848,1182,1560 K,0.032,190.0000,1.53,13000,1670,1.57,287,1.12,"[9.322699, 18.21115, 153.896205, 217.7185861]",-0.520000
B,10.811000,5,0.85,0.87,4200,320.0,[3],no data,2460,1851,2349 K,NaN,27.0000,1.92,16200,49000,2.04,NaN,no data,"[8.298019, 25.15483, 37.93059, 259.3715, 340.2...",0.279723
C,12.010700,6,0.70,0.67,4300,33.0,"[-4, 4]",no data,2267,500,3800 K,NaN,140.0000,1.70,18350,NaN,2.55,NaN,no data,"[11.260288, 24.383154, 47.88778, 64.49352, 392...",1.262114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pb,207.200000,82,1.80,1.54,2022,46.0,"[2, 4]",no data,11340,1421.39,600.61 K,0.440,35.0000,2.02,1260,NaN,2.33,16,1.75,"[7.4166799, 15.032499, 31.9373, 42.33256, 68.8...",0.356721
Bi,208.980400,83,1.60,1.43,1837,31.0,[3],no data,9780,1292.6,544.4 K,0.330,8.0000,2.07,1790,NaN,2.02,32,1.82,"[7.285516, 16.703, 25.57075, 45.37, 54.856, 88...",0.942362
Ac,227.000000,89,1.95,no data,3573,NaN,[3],no data,10070,2250,1323 K,NaN,12.0000,2.47,NaN,NaN,1.10,NaN,1.878,"[5.380226, 11.75, 17.431, 44.8, 55.0, 67.0, 79...",0.350000
Th,232.038060,90,1.80,no data,5093,54.0,[4],no data,11724,2978,2115 K,0.270,54.0000,2.45,2490,350,1.30,79,1.798,"[6.3067, 12.1, 18.32, 28.648, 58.0, 69.1, 82.0...",1.170000


In [14]:
cprops_df['Melting point'] = cprops_df['Melting point'].str.replace("K", "")
cprops_df['Melting point'] = cprops_df['Melting point'].str.replace("white P", "")
cprops_df['Melting point'] = cprops_df['Melting point'].str.replace(r"\(.*\)","",  regex=True).astype(float)
#cprops_df['Melting point'] = [float(a.replace('K', '', 1)) for a in cprops_df['Melting point']]

In [15]:
cprops_df

,Atomic mass,Atomic no,Atomic radius,Atomic radius calculated,Boiling point,Bulk modulus,Common oxidation states,Critical temperature,Density of solid,Liquid range,Melting point,Poissons ratio,Thermal conductivity,Van der waals radius,Velocity of sound,Vickers hardness,X,Youngs modulus,Metallic radius,Ionization energies,Electron affinity
H,1.007940,1,0.25,0.53,20.28,NaN,"[-1, 1]",33,NaN,6.27,14.01,NaN,0.1805,1.10,1270,NaN,2.20,NaN,no data,[13.598434599702],0.754598
Li,6.941000,3,1.45,1.67,1615,11.0,[1],3223,535,1161.31,453.69,NaN,85.0000,1.82,6000,NaN,0.98,4.9,1.52,"[5.391714996, 75.640097, 122.45435914]",0.618049
Be,9.012182,4,1.05,1.12,2742,130.0,[2],no data,1848,1182,1560.00,0.032,190.0000,1.53,13000,1670,1.57,287,1.12,"[9.322699, 18.21115, 153.896205, 217.7185861]",-0.520000
B,10.811000,5,0.85,0.87,4200,320.0,[3],no data,2460,1851,2349.00,NaN,27.0000,1.92,16200,49000,2.04,NaN,no data,"[8.298019, 25.15483, 37.93059, 259.3715, 340.2...",0.279723
C,12.010700,6,0.70,0.67,4300,33.0,"[-4, 4]",no data,2267,500,3800.00,NaN,140.0000,1.70,18350,NaN,2.55,NaN,no data,"[11.260288, 24.383154, 47.88778, 64.49352, 392...",1.262114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pb,207.200000,82,1.80,1.54,2022,46.0,"[2, 4]",no data,11340,1421.39,600.61,0.440,35.0000,2.02,1260,NaN,2.33,16,1.75,"[7.4166799, 15.032499, 31.9373, 42.33256, 68.8...",0.356721
Bi,208.980400,83,1.60,1.43,1837,31.0,[3],no data,9780,1292.6,544.40,0.330,8.0000,2.07,1790,NaN,2.02,32,1.82,"[7.285516, 16.703, 25.57075, 45.37, 54.856, 88...",0.942362
Ac,227.000000,89,1.95,no data,3573,NaN,[3],no data,10070,2250,1323.00,NaN,12.0000,2.47,NaN,NaN,1.10,NaN,1.878,"[5.380226, 11.75, 17.431, 44.8, 55.0, 67.0, 79...",0.350000
Th,232.038060,90,1.80,no data,5093,54.0,[4],no data,11724,2978,2115.00,0.270,54.0000,2.45,2490,350,1.30,79,1.798,"[6.3067, 12.1, 18.32, 28.648, 58.0, 69.1, 82.0...",1.170000


In [16]:
cprops_df['Metallic radius'] = cprops_df['Metallic radius'].astype(str)
cprops_df['Metallic radius'] = [(a.replace('no data', 'NaN', 1)) for a in cprops_df['Metallic radius']]
cprops_df['Metallic radius'] = cprops_df['Metallic radius'].astype(float)
cprops_df


,Atomic mass,Atomic no,Atomic radius,Atomic radius calculated,Boiling point,Bulk modulus,Common oxidation states,Critical temperature,Density of solid,Liquid range,Melting point,Poissons ratio,Thermal conductivity,Van der waals radius,Velocity of sound,Vickers hardness,X,Youngs modulus,Metallic radius,Ionization energies,Electron affinity
H,1.007940,1,0.25,0.53,20.28,NaN,"[-1, 1]",33,NaN,6.27,14.01,NaN,0.1805,1.10,1270,NaN,2.20,NaN,NaN,[13.598434599702],0.754598
Li,6.941000,3,1.45,1.67,1615,11.0,[1],3223,535,1161.31,453.69,NaN,85.0000,1.82,6000,NaN,0.98,4.9,1.520,"[5.391714996, 75.640097, 122.45435914]",0.618049
Be,9.012182,4,1.05,1.12,2742,130.0,[2],no data,1848,1182,1560.00,0.032,190.0000,1.53,13000,1670,1.57,287,1.120,"[9.322699, 18.21115, 153.896205, 217.7185861]",-0.520000
B,10.811000,5,0.85,0.87,4200,320.0,[3],no data,2460,1851,2349.00,NaN,27.0000,1.92,16200,49000,2.04,NaN,NaN,"[8.298019, 25.15483, 37.93059, 259.3715, 340.2...",0.279723
C,12.010700,6,0.70,0.67,4300,33.0,"[-4, 4]",no data,2267,500,3800.00,NaN,140.0000,1.70,18350,NaN,2.55,NaN,NaN,"[11.260288, 24.383154, 47.88778, 64.49352, 392...",1.262114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pb,207.200000,82,1.80,1.54,2022,46.0,"[2, 4]",no data,11340,1421.39,600.61,0.440,35.0000,2.02,1260,NaN,2.33,16,1.750,"[7.4166799, 15.032499, 31.9373, 42.33256, 68.8...",0.356721
Bi,208.980400,83,1.60,1.43,1837,31.0,[3],no data,9780,1292.6,544.40,0.330,8.0000,2.07,1790,NaN,2.02,32,1.820,"[7.285516, 16.703, 25.57075, 45.37, 54.856, 88...",0.942362
Ac,227.000000,89,1.95,no data,3573,NaN,[3],no data,10070,2250,1323.00,NaN,12.0000,2.47,NaN,NaN,1.10,NaN,1.878,"[5.380226, 11.75, 17.431, 44.8, 55.0, 67.0, 79...",0.350000
Th,232.038060,90,1.80,no data,5093,54.0,[4],no data,11724,2978,2115.00,0.270,54.0000,2.45,2490,350,1.30,79,1.798,"[6.3067, 12.1, 18.32, 28.648, 58.0, 69.1, 82.0...",1.170000


In [17]:
type(cprops_df['Metallic radius'][1])



numpy.float64

In [18]:
#cprops_df['Common oxidation states'] = [len(a) for a in cprops_df['Common oxidation states']]
#cprops_df['Common oxidation states'] 

#executivejojodecision 841 11:30

In [19]:
type(cprops_df['Ionization energies'][0])

list

In [20]:
cprops_df['First Ionization Energy'] = [a[0] for a in cprops_df['Ionization energies']]

In [21]:
cprops_df

,Atomic mass,Atomic no,Atomic radius,Atomic radius calculated,Boiling point,Bulk modulus,Common oxidation states,Critical temperature,Density of solid,Liquid range,Melting point,Poissons ratio,Thermal conductivity,Van der waals radius,Velocity of sound,Vickers hardness,X,Youngs modulus,Metallic radius,Ionization energies,Electron affinity,First Ionization Energy
H,1.007940,1,0.25,0.53,20.28,NaN,"[-1, 1]",33,NaN,6.27,14.01,NaN,0.1805,1.10,1270,NaN,2.20,NaN,NaN,[13.598434599702],0.754598,13.598435
Li,6.941000,3,1.45,1.67,1615,11.0,[1],3223,535,1161.31,453.69,NaN,85.0000,1.82,6000,NaN,0.98,4.9,1.520,"[5.391714996, 75.640097, 122.45435914]",0.618049,5.391715
Be,9.012182,4,1.05,1.12,2742,130.0,[2],no data,1848,1182,1560.00,0.032,190.0000,1.53,13000,1670,1.57,287,1.120,"[9.322699, 18.21115, 153.896205, 217.7185861]",-0.520000,9.322699
B,10.811000,5,0.85,0.87,4200,320.0,[3],no data,2460,1851,2349.00,NaN,27.0000,1.92,16200,49000,2.04,NaN,NaN,"[8.298019, 25.15483, 37.93059, 259.3715, 340.2...",0.279723,8.298019
C,12.010700,6,0.70,0.67,4300,33.0,"[-4, 4]",no data,2267,500,3800.00,NaN,140.0000,1.70,18350,NaN,2.55,NaN,NaN,"[11.260288, 24.383154, 47.88778, 64.49352, 392...",1.262114,11.260288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pb,207.200000,82,1.80,1.54,2022,46.0,"[2, 4]",no data,11340,1421.39,600.61,0.440,35.0000,2.02,1260,NaN,2.33,16,1.750,"[7.4166799, 15.032499, 31.9373, 42.33256, 68.8...",0.356721,7.416680
Bi,208.980400,83,1.60,1.43,1837,31.0,[3],no data,9780,1292.6,544.40,0.330,8.0000,2.07,1790,NaN,2.02,32,1.820,"[7.285516, 16.703, 25.57075, 45.37, 54.856, 88...",0.942362,7.285516
Ac,227.000000,89,1.95,no data,3573,NaN,[3],no data,10070,2250,1323.00,NaN,12.0000,2.47,NaN,NaN,1.10,NaN,1.878,"[5.380226, 11.75, 17.431, 44.8, 55.0, 67.0, 79...",0.350000,5.380226
Th,232.038060,90,1.80,no data,5093,54.0,[4],no data,11724,2978,2115.00,0.270,54.0000,2.45,2490,350,1.30,79,1.798,"[6.3067, 12.1, 18.32, 28.648, 58.0, 69.1, 82.0...",1.170000,6.306700


In [22]:
cprops_df = cprops_df.drop("Ionization energies", axis=1)

In [23]:
cprops_df

,Atomic mass,Atomic no,Atomic radius,Atomic radius calculated,Boiling point,Bulk modulus,Common oxidation states,Critical temperature,Density of solid,Liquid range,Melting point,Poissons ratio,Thermal conductivity,Van der waals radius,Velocity of sound,Vickers hardness,X,Youngs modulus,Metallic radius,Electron affinity,First Ionization Energy
H,1.007940,1,0.25,0.53,20.28,NaN,"[-1, 1]",33,NaN,6.27,14.01,NaN,0.1805,1.10,1270,NaN,2.20,NaN,NaN,0.754598,13.598435
Li,6.941000,3,1.45,1.67,1615,11.0,[1],3223,535,1161.31,453.69,NaN,85.0000,1.82,6000,NaN,0.98,4.9,1.520,0.618049,5.391715
Be,9.012182,4,1.05,1.12,2742,130.0,[2],no data,1848,1182,1560.00,0.032,190.0000,1.53,13000,1670,1.57,287,1.120,-0.520000,9.322699
B,10.811000,5,0.85,0.87,4200,320.0,[3],no data,2460,1851,2349.00,NaN,27.0000,1.92,16200,49000,2.04,NaN,NaN,0.279723,8.298019
C,12.010700,6,0.70,0.67,4300,33.0,"[-4, 4]",no data,2267,500,3800.00,NaN,140.0000,1.70,18350,NaN,2.55,NaN,NaN,1.262114,11.260288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pb,207.200000,82,1.80,1.54,2022,46.0,"[2, 4]",no data,11340,1421.39,600.61,0.440,35.0000,2.02,1260,NaN,2.33,16,1.750,0.356721,7.416680
Bi,208.980400,83,1.60,1.43,1837,31.0,[3],no data,9780,1292.6,544.40,0.330,8.0000,2.07,1790,NaN,2.02,32,1.820,0.942362,7.285516
Ac,227.000000,89,1.95,no data,3573,NaN,[3],no data,10070,2250,1323.00,NaN,12.0000,2.47,NaN,NaN,1.10,NaN,1.878,0.350000,5.380226
Th,232.038060,90,1.80,no data,5093,54.0,[4],no data,11724,2978,2115.00,0.270,54.0000,2.45,2490,350,1.30,79,1.798,1.170000,6.306700


In [24]:
cprops_df['Critical temperature'] = [(a.replace('no data', 'NaN', 1)) for a in cprops_df['Critical temperature']]
cprops_df

,Atomic mass,Atomic no,Atomic radius,Atomic radius calculated,Boiling point,Bulk modulus,Common oxidation states,Critical temperature,Density of solid,Liquid range,Melting point,Poissons ratio,Thermal conductivity,Van der waals radius,Velocity of sound,Vickers hardness,X,Youngs modulus,Metallic radius,Electron affinity,First Ionization Energy
H,1.007940,1,0.25,0.53,20.28,NaN,"[-1, 1]",33,NaN,6.27,14.01,NaN,0.1805,1.10,1270,NaN,2.20,NaN,NaN,0.754598,13.598435
Li,6.941000,3,1.45,1.67,1615,11.0,[1],3223,535,1161.31,453.69,NaN,85.0000,1.82,6000,NaN,0.98,4.9,1.520,0.618049,5.391715
Be,9.012182,4,1.05,1.12,2742,130.0,[2],NaN,1848,1182,1560.00,0.032,190.0000,1.53,13000,1670,1.57,287,1.120,-0.520000,9.322699
B,10.811000,5,0.85,0.87,4200,320.0,[3],NaN,2460,1851,2349.00,NaN,27.0000,1.92,16200,49000,2.04,NaN,NaN,0.279723,8.298019
C,12.010700,6,0.70,0.67,4300,33.0,"[-4, 4]",NaN,2267,500,3800.00,NaN,140.0000,1.70,18350,NaN,2.55,NaN,NaN,1.262114,11.260288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pb,207.200000,82,1.80,1.54,2022,46.0,"[2, 4]",NaN,11340,1421.39,600.61,0.440,35.0000,2.02,1260,NaN,2.33,16,1.750,0.356721,7.416680
Bi,208.980400,83,1.60,1.43,1837,31.0,[3],NaN,9780,1292.6,544.40,0.330,8.0000,2.07,1790,NaN,2.02,32,1.820,0.942362,7.285516
Ac,227.000000,89,1.95,no data,3573,NaN,[3],NaN,10070,2250,1323.00,NaN,12.0000,2.47,NaN,NaN,1.10,NaN,1.878,0.350000,5.380226
Th,232.038060,90,1.80,no data,5093,54.0,[4],NaN,11724,2978,2115.00,0.270,54.0000,2.45,2490,350,1.30,79,1.798,1.170000,6.306700


In [25]:
# OG_names = ['AtomicRadius', 'AtomicVolume', 'AtomicWeight', 'BulkModulus',
#        'BoilingT', 'Column', 'CovalentRadius', 'Density', 'ElectronAffinity',
#        'Electronegativity', 'FirstIonizationEnergy', 'HeatCapacityMass', 'Row',
#        'phi', 'SecondIonizationEnergy', 'ShearModulus', 'Superconduction temperature', 'Velocity of sound']



In [26]:
cprops_df.dtypes

Atomic mass                 float64
Atomic no                     int64
Atomic radius               float64
Atomic radius calculated     object
Boiling point                object
Bulk modulus                float64
Common oxidation states      object
Critical temperature         object
Density of solid             object
Liquid range                 object
Melting point               float64
Poissons ratio              float64
Thermal conductivity        float64
Van der waals radius        float64
Velocity of sound            object
Vickers hardness             object
X                           float64
Youngs modulus               object
Metallic radius             float64
Electron affinity           float64
First Ionization Energy     float64
dtype: object

In [27]:
cprops_df = cprops_df.apply(pd.to_numeric, errors='coerce')

In [28]:
cprops_df.dtypes

Atomic mass                 float64
Atomic no                     int64
Atomic radius               float64
Atomic radius calculated    float64
Boiling point               float64
Bulk modulus                float64
Common oxidation states     float64
Critical temperature        float64
Density of solid            float64
Liquid range                float64
Melting point               float64
Poissons ratio              float64
Thermal conductivity        float64
Van der waals radius        float64
Velocity of sound           float64
Vickers hardness            float64
X                           float64
Youngs modulus              float64
Metallic radius             float64
Electron affinity           float64
First Ionization Energy     float64
dtype: object

In [29]:
# Computing mean values using mean function 
mvals = dict(cprops_df.mean())
mvals

{'Atomic mass': 107.06698397475002,
 'Atomic no': 44.7,
 'Atomic radius': 1.479999999999999,
 'Atomic radius calculated': 1.6537837837837837,
 'Boiling point': 2696.401625,
 'Bulk modulus': 91.6441176470588,
 'Common oxidation states': nan,
 'Critical temperature': 1285.5176470588235,
 'Density of solid': 8067.890410958904,
 'Liquid range': 1317.8828749999998,
 'Melting point': 1383.99375,
 'Poissons ratio': 0.2942692307692307,
 'Thermal conductivity': 66.761243875,
 'Van der waals radius': 2.1264999999999996,
 'Velocity of sound': 3723.0015151515154,
 'Vickers hardness': 2108.3076923076924,
 'X': 1.7591249999999998,
 'Youngs modulus': 111.78688524590164,
 'Metallic radius': 1.622590909090909,
 'Electron affinity': 0.7264296744124998,
 'First Ionization Energy': 7.683228095414148}

In [30]:
# Iterating through variable with averages to replace the NaN values in element_data
for key, value in mvals.items():
    cprops_df.loc[cprops_df[key].isnull(),key] = value

display(cprops_df)

,Atomic mass,Atomic no,Atomic radius,Atomic radius calculated,Boiling point,Bulk modulus,Common oxidation states,Critical temperature,Density of solid,Liquid range,Melting point,Poissons ratio,Thermal conductivity,Van der waals radius,Velocity of sound,Vickers hardness,X,Youngs modulus,Metallic radius,Electron affinity,First Ionization Energy
H,1.007940,1.0,0.25,0.530000,20.28,91.644118,NaN,33.000000,8067.890411,6.27,14.01,0.294269,0.1805,1.10,1270.000000,2108.307692,2.20,111.786885,1.622591,0.754598,13.598435
Li,6.941000,3.0,1.45,1.670000,1615.00,11.000000,NaN,3223.000000,535.000000,1161.31,453.69,0.294269,85.0000,1.82,6000.000000,2108.307692,0.98,4.900000,1.520000,0.618049,5.391715
Be,9.012182,4.0,1.05,1.120000,2742.00,130.000000,NaN,1285.517647,1848.000000,1182.00,1560.00,0.032000,190.0000,1.53,13000.000000,1670.000000,1.57,287.000000,1.120000,-0.520000,9.322699
B,10.811000,5.0,0.85,0.870000,4200.00,320.000000,NaN,1285.517647,2460.000000,1851.00,2349.00,0.294269,27.0000,1.92,16200.000000,49000.000000,2.04,111.786885,1.622591,0.279723,8.298019
C,12.010700,6.0,0.70,0.670000,4300.00,33.000000,NaN,1285.517647,2267.000000,500.00,3800.00,0.294269,140.0000,1.70,18350.000000,2108.307692,2.55,111.786885,1.622591,1.262114,11.260288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pb,207.200000,82.0,1.80,1.540000,2022.00,46.000000,NaN,1285.517647,11340.000000,1421.39,600.61,0.440000,35.0000,2.02,1260.000000,2108.307692,2.33,16.000000,1.750000,0.356721,7.416680
Bi,208.980400,83.0,1.60,1.430000,1837.00,31.000000,NaN,1285.517647,9780.000000,1292.60,544.40,0.330000,8.0000,2.07,1790.000000,2108.307692,2.02,32.000000,1.820000,0.942362,7.285516
Ac,227.000000,89.0,1.95,1.653784,3573.00,91.644118,NaN,1285.517647,10070.000000,2250.00,1323.00,0.294269,12.0000,2.47,3723.001515,2108.307692,1.10,111.786885,1.878000,0.350000,5.380226
Th,232.038060,90.0,1.80,1.653784,5093.00,54.000000,NaN,1285.517647,11724.000000,2978.00,2115.00,0.270000,54.0000,2.45,2490.000000,350.000000,1.30,79.000000,1.798000,1.170000,6.306700


In [31]:
# # Initializing variables
# Kp3_2 = list(filt_data["pretty_formula"])
# CFp3_2 = []
# task_id = filt_data["material_id"]

# properties = list(cprops_df.columns)
# compound_names  =  list(filt_data["pretty_formula"])
# average_dict = {}
# max_dict = {}
# min_dict = {}

# # Implementing same method as question 4 to compute max, min, and averages for all properties in data
# for elms in range(len(Kp3_2)): 
#     #Kp3_2 = filt_data["pretty_formula"]
#     #task = task_id[elms]
#     Kp3_2 = filt_data["pretty_formula"]
#     comp_eli_2 = Composition(CFp3_2)
#     CEl_2 = comp_eli_2.elements
#     q2dict = comp_eli_2.to_data_dict['unit_cell_composition']
#     row_avgs = {}
#     row_max = {}
#     row_min = {}
    
#     for col in properties:
#       overall = 0
#       allval = []
#       for key, value in q2dict.items():
#         property_ = cprops_df.loc[key, col]
#         overall += property_ * value
#         allval.append(property_)
#       num_total = sum(q2dict.values())
#       avg = overall/num_total
#       row_avgs[col + '_avg'] = avg
#       row_max[col + '_max'] = max(allval)
#       row_min[col + '_min'] = min(allval)
#     average_dict[task] = row_avgs
#     max_dict[task] = row_max
#     min_dict[task] = row_min
    

In [33]:
#jojoversion

In [34]:
filt_data_comp = filt_data
filt_data_comp['Composition'] = [Composition(c) for c in filt_data_comp["pretty_formula"]]
filt_data_comp

,material_id,energy,energy_per_atom,volume,formation_energy_per_atom,nsites,pretty_formula,nelements,density,band_gap,Composition
0,mp-1001034,-52.019078,-3.715648,376.145863,-0.703661,14,Mg(InSe2)2,3,5.030727,0.7432,"(Mg, In, Se)"
1,mp-1001780,-22.364406,-5.591101,78.515319,-1.699567,4,LuCuS2,3,6.400668,1.5031,"(Lu, Cu, S)"
2,mp-1001786,-23.600913,-5.900228,71.701237,-2.103385,4,LiScS2,3,2.687084,1.5296,"(Li, Sc, S)"
3,mp-1002124,-19.780759,-9.890380,32.055765,-0.298402,2,HfC,2,9.868236,0.5774,"(Hf, C)"
4,mp-1004528,-95.454964,-5.614998,324.979430,-3.162482,17,CsB3PbF12,4,3.068458,6.3125,"(Cs, B, Pb, F)"
...,...,...,...,...,...,...,...,...,...,...,...
5614,mvc-6916,-99.385289,-7.098949,156.686110,-2.307339,14,MgMn2O4,3,4.200544,0.0516,"(Mg, Mn, O)"
5615,mvc-6928,-91.603914,-6.543137,173.941832,-2.192463,14,Ca(FeO2)2,3,4.119621,1.7463,"(Ca, Fe, O)"
5616,mvc-6946,-36.735653,-6.122609,97.529160,-1.961400,6,SnO2,2,5.131960,2.1009,"(Sn, O)"
5617,mvc-7040,-145.212422,-8.067357,418.920727,-1.999722,18,Re2O7,2,3.840256,3.4689,"(Re, O)"


In [35]:
cprops_df_dict = cprops_df.to_dict()

rad_dict = cprops_df_dict['Atomic radius']

def radius_mean(composition):
    sumofradii = 0
    totnumatoms = 0
    for element, number in composition.items():
        sumofradii += (number*rad_dict[str(element)])
        totnumatoms += number
    return sumofradii/totnumatoms

atomic_radius = filt_data_comp['Composition'].apply(radius_mean)
atomic_radius

0       1.314286
1       1.275000
2       1.262500
3       1.125000
4       0.761765
          ...   
5614    0.957143
5615    1.000000
5616    0.883333
5617    0.766667
5618    0.835000
Name: Composition, Length: 5614, dtype: float64

In [36]:
#jojo version

def propertymean(property, composition):
    sumofproperty = 0
    totalnumatoms = 0
    for element, number in composition.items():
        sumofproperty += (number*cprops_df_dict[property][str(element)])
        totalnumatoms += number
    return sumofproperty/totalnumatoms

def maxofproperty(property, composition):
    propmax = None
    for element, number in composition.items():
        propertyvalue = cprops_df_dict[property][str(element)]
        if propmax:
            propmax = propertyvalue if propertyvalue > propmax else propmax
        else:
            propmax = propertyvalue
    return propmax

def minofproperty(property, composition):
    propmin = None
    for element, number in composition.items():
        propertyvalue = cprops_df_dict[property][str(element)]
        if propmin:
            propmin = propertyvalue if propertyvalue < propmin else propmin
        else:
            propmin = propertyvalue
    return propmin

In [37]:
avg_properties_df = pd.DataFrame()

for property in cprops_df.columns:
    individualpropertymean = partial(propertymean, property)
    averages = filt_data_comp['Composition'].apply(individualpropertymean)
    avg_properties_df[("average_" + property)] = averages
    
avg_properties_df.head()
print("Average properties Dimension: ", avg_properties_df.shape)

Average properties Dimension:  (5614, 21)


In [38]:
max_properties = pd.DataFrame()

for property in cprops_df.columns:
    individualpropertymax = partial(maxofproperty, property)
    max = filt_data_comp['Composition'].apply(individualpropertymax)
    max_properties[("max_" + property)] = max

In [39]:
min_properties = pd.DataFrame()

for property in cprops_df.columns:
    individualpropertymin = partial(minofproperty, property)
    min = filt_data_comp['Composition'].apply(individualpropertymin)
    min_properties[("min_" + property)] = min

In [40]:
avg_properties_df

,average_Atomic mass,average_Atomic no,average_Atomic radius,average_Atomic radius calculated,average_Boiling point,average_Bulk modulus,average_Common oxidation states,average_Critical temperature,average_Density of solid,average_Liquid range,average_Melting point,average_Poissons ratio,average_Thermal conductivity,average_Van der waals radius,average_Velocity of sound,average_Vickers hardness,average_X,average_Youngs modulus,average_Metallic radius,average_Electron affinity,average_First Ionization Energy
0,81.397286,35.142857,1.314286,1.241429,1412.142857,37.355462,NaN,1560.078992,5090.571429,875.214286,536.928571,0.314077,46.582857,1.884286,2918.857143,2108.307692,2.152857,15.285714,1.632909,1.204324,8.318359
1,75.660750,33.000000,1.275000,1.345000,2077.685000,50.850000,NaN,1299.758824,5670.250000,1062.812500,1014.872500,0.297135,104.102500,1.950000,3684.751136,1436.403846,2.082500,105.643443,1.564545,1.407216,8.468068
2,29.006728,14.000000,1.262500,1.317500,1538.435000,20.850000,NaN,1784.129412,1860.000000,777.332500,761.102500,0.290702,25.352500,1.892500,4292.251136,2108.307692,1.875000,75.618443,1.601545,1.240115,8.168306
3,95.250350,39.000000,1.125000,1.375000,4588.000000,71.500000,NaN,1285.517647,7788.500000,1435.000000,3153.000000,0.332135,81.500000,1.965000,10680.000000,1934.153846,1.925000,94.893443,1.601295,0.720092,9.042679
4,35.324664,15.294118,0.761765,0.715882,975.668235,123.960554,NaN,518.121799,6906.687349,470.282353,505.385882,0.302842,8.960729,1.697059,5779.942335,10383.312217,3.352941,99.676663,1.694580,2.498929,14.428146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5614,28.311241,13.428571,0.957143,0.941429,913.114286,93.082353,NaN,639.278992,6992.794521,315.942857,597.171429,0.293659,25.100903,1.701429,2310.285714,2108.307692,2.595714,126.878220,1.524909,0.626346,10.998076
5615,30.823657,14.857143,1.000000,0.997143,1197.971429,103.368067,NaN,639.278992,7081.365949,489.942857,708.028571,0.295297,51.443760,1.781429,2128.571429,1679.648352,2.631429,127.021077,1.574338,0.882206,10.912902
5616,50.236267,22.000000,0.883333,0.803333,1018.466667,80.429412,NaN,531.572549,7815.260274,813.573333,204.893333,0.316179,22.351053,1.736667,1045.000000,2108.307692,2.946667,91.191257,1.608394,1.344760,11.526676
5617,53.823311,22.888889,0.766667,0.791111,1374.377778,153.500980,NaN,405.915033,10946.136986,563.088889,811.288889,0.295543,10.687340,1.662222,1291.388889,2184.239316,3.097778,189.834244,1.567571,1.149837,12.332603


In [41]:
min_properties


,min_Atomic mass,min_Atomic no,min_Atomic radius,min_Atomic radius calculated,min_Boiling point,min_Bulk modulus,min_Common oxidation states,min_Critical temperature,min_Density of solid,min_Liquid range,min_Melting point,min_Poissons ratio,min_Thermal conductivity,min_Van der waals radius,min_Velocity of sound,min_Vickers hardness,min_X,min_Youngs modulus,min_Metallic radius,min_Electron affinity,min_First Ionization Energy
0,24.3050,12.0,1.15,1.03,958.00,8.300000,NaN,1285.517647,1738.000000,440.00,429.75,0.290000,0.52000,1.73,1215.000000,2108.307692,1.31,10.000000,1.600000,-0.420000,5.786356
1,32.0650,16.0,1.00,0.88,717.87,7.700000,NaN,1285.517647,1960.000000,329.51,388.36,0.260000,0.20500,1.80,3570.000000,369.000000,1.27,69.000000,1.278000,0.238870,5.425871
2,6.9410,3.0,1.00,0.88,717.87,7.700000,NaN,1285.517647,535.000000,329.51,388.36,0.280000,0.20500,1.80,3723.001515,2108.307692,0.98,4.900000,1.520000,0.188200,5.391715
3,12.0107,6.0,0.70,0.67,4300.00,33.000000,NaN,1285.517647,2267.000000,500.00,2506.00,0.294269,23.00000,1.70,3010.000000,1760.000000,1.30,78.000000,1.580000,0.178070,6.825070
4,10.8110,5.0,0.50,0.42,85.03,1.600000,NaN,144.000000,1879.000000,31.50,53.53,0.294269,0.02770,1.47,1260.000000,2108.307692,0.79,1.700000,1.622591,0.279723,3.893906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5614,15.9994,8.0,0.60,0.48,90.20,45.000000,NaN,154.600000,1738.000000,35.40,54.80,0.290000,0.02658,1.52,317.500000,2108.307692,1.31,45.000000,1.292000,-0.520000,7.434038
5615,15.9994,8.0,0.60,0.48,90.20,17.000000,NaN,154.600000,1550.000000,35.40,54.80,0.290000,0.02658,1.52,317.500000,608.000000,1.00,20.000000,1.277000,0.024551,6.113155
5616,15.9994,8.0,0.60,0.48,90.20,58.000000,NaN,154.600000,7310.000000,35.40,54.80,0.294269,0.02658,1.52,317.500000,2108.307692,1.96,50.000000,1.580000,1.112070,7.343918
5617,15.9994,8.0,0.60,0.48,90.20,91.644118,NaN,154.600000,8067.890411,35.40,54.80,0.294269,0.02658,1.52,317.500000,2108.307692,1.90,111.786885,1.375000,0.060397,7.833520


In [42]:
max_properties

,max_Atomic mass,max_Atomic no,max_Atomic radius,max_Atomic radius calculated,max_Boiling point,max_Bulk modulus,max_Common oxidation states,max_Critical temperature,max_Density of solid,max_Liquid range,max_Melting point,max_Poissons ratio,max_Thermal conductivity,max_Van der waals radius,max_Velocity of sound,max_Vickers hardness,max_X,max_Youngs modulus,max_Metallic radius,max_Electron affinity,max_First Ionization Energy
0,114.818000,49.0,1.55,1.56,2345.0,91.644118,NaN,1766.000000,7310.000000,1915.25,923.00,0.330000,160.0,1.93,4602.000000,2108.307692,2.55,45.000000,1.670000,2.020605,9.752392
1,174.967000,71.0,1.75,2.17,3675.0,140.000000,NaN,1314.000000,9841.000000,1842.23,1925.00,0.340000,400.0,2.24,3723.001515,2108.307692,2.58,130.000000,1.735000,2.077105,10.360010
2,44.955912,21.0,1.60,1.84,3103.0,57.000000,NaN,3223.000000,2985.000000,1289.00,1814.00,0.294269,85.0,2.15,6000.000000,2108.307692,2.58,111.786885,1.641000,2.077105,10.360010
3,178.490000,72.0,1.55,2.08,4876.0,110.000000,NaN,1285.517647,13310.000000,2370.00,3800.00,0.370000,140.0,2.23,18350.000000,2108.307692,2.55,111.786885,1.622591,1.262114,11.260288
4,207.200000,82.0,2.60,2.98,4200.0,320.000000,NaN,1938.000000,11340.000000,1851.00,2349.00,0.440000,36.0,3.43,16200.000000,49000.000000,3.98,111.786885,2.719000,3.401190,17.422820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5614,54.938045,25.0,1.50,1.61,2334.0,120.000000,NaN,1285.517647,8067.890411,815.00,1519.00,0.294269,160.0,2.05,5150.000000,2108.307692,3.44,198.000000,1.622591,1.461105,13.618055
5615,55.845000,26.0,1.80,1.94,3134.0,170.000000,NaN,1285.517647,8067.890411,1323.00,1811.00,0.310000,200.0,2.31,4910.000000,2108.307692,3.44,211.000000,1.976000,1.461105,13.618055
5616,118.710000,50.0,1.45,1.45,2875.0,91.644118,NaN,1285.517647,8067.890411,2369.92,505.08,0.360000,67.0,2.17,2500.000000,2108.307692,3.44,111.786885,1.622591,1.461105,13.618055
5617,186.207000,75.0,1.35,1.88,5869.0,370.000000,NaN,1285.517647,21020.000000,2410.00,3459.00,0.300000,48.0,2.16,4700.000000,2450.000000,3.44,463.000000,1.622591,1.461105,13.618055


In [56]:
design_matrix= pd.concat([filt_data, avg_properties_df, min_properties, max_properties], axis=1)
design_matrix

,material_id,energy,energy_per_atom,volume,formation_energy_per_atom,nsites,pretty_formula,nelements,density,band_gap,Composition,average_Atomic mass,average_Atomic no,average_Atomic radius,average_Atomic radius calculated,average_Boiling point,average_Bulk modulus,average_Common oxidation states,average_Critical temperature,average_Density of solid,average_Liquid range,average_Melting point,average_Poissons ratio,average_Thermal conductivity,average_Van der waals radius,average_Velocity of sound,average_Vickers hardness,average_X,average_Youngs modulus,average_Metallic radius,average_Electron affinity,average_First Ionization Energy,min_Atomic mass,min_Atomic no,min_Atomic radius,min_Atomic radius calculated,min_Boiling point,min_Bulk modulus,min_Common oxidation states,min_Critical temperature,min_Density of solid,min_Liquid range,min_Melting point,min_Poissons ratio,min_Thermal conductivity,min_Van der waals radius,min_Velocity of sound,min_Vickers hardness,min_X,min_Youngs modulus,min_Metallic radius,min_Electron affinity,min_First Ionization Energy,max_Atomic mass,max_Atomic no,max_Atomic radius,max_Atomic radius calculated,max_Boiling point,max_Bulk modulus,max_Common oxidation states,max_Critical temperature,max_Density of solid,max_Liquid range,max_Melting point,max_Poissons ratio,max_Thermal conductivity,max_Van der waals radius,max_Velocity of sound,max_Vickers hardness,max_X,max_Youngs modulus,max_Metallic radius,max_Electron affinity,max_First Ionization Energy
0,mp-1001034,-52.019078,-3.715648,376.145863,-0.703661,14,Mg(InSe2)2,3,5.030727,0.7432,"(Mg, In, Se)",81.397286,35.142857,1.314286,1.241429,1412.142857,37.355462,NaN,1560.078992,5090.571429,875.214286,536.928571,0.314077,46.582857,1.884286,2918.857143,2108.307692,2.152857,15.285714,1.632909,1.204324,8.318359,24.3050,12.0,1.15,1.03,958.00,8.300000,NaN,1285.517647,1738.000000,440.00,429.75,0.290000,0.52000,1.73,1215.000000,2108.307692,1.31,10.000000,1.600000,-0.420000,5.786356,114.818000,49.0,1.55,1.56,2345.0,91.644118,NaN,1766.000000,7310.000000,1915.25,923.00,0.330000,160.0,1.93,4602.000000,2108.307692,2.55,45.000000,1.670000,2.020605,9.752392
1,mp-1001780,-22.364406,-5.591101,78.515319,-1.699567,4,LuCuS2,3,6.400668,1.5031,"(Lu, Cu, S)",75.660750,33.000000,1.275000,1.345000,2077.685000,50.850000,NaN,1299.758824,5670.250000,1062.812500,1014.872500,0.297135,104.102500,1.950000,3684.751136,1436.403846,2.082500,105.643443,1.564545,1.407216,8.468068,32.0650,16.0,1.00,0.88,717.87,7.700000,NaN,1285.517647,1960.000000,329.51,388.36,0.260000,0.20500,1.80,3570.000000,369.000000,1.27,69.000000,1.278000,0.238870,5.425871,174.967000,71.0,1.75,2.17,3675.0,140.000000,NaN,1314.000000,9841.000000,1842.23,1925.00,0.340000,400.0,2.24,3723.001515,2108.307692,2.58,130.000000,1.735000,2.077105,10.360010
2,mp-1001786,-23.600913,-5.900228,71.701237,-2.103385,4,LiScS2,3,2.687084,1.5296,"(Li, Sc, S)",29.006728,14.000000,1.262500,1.317500,1538.435000,20.850000,NaN,1784.129412,1860.000000,777.332500,761.102500,0.290702,25.352500,1.892500,4292.251136,2108.307692,1.875000,75.618443,1.601545,1.240115,8.168306,6.9410,3.0,1.00,0.88,717.87,7.700000,NaN,1285.517647,535.000000,329.51,388.36,0.280000,0.20500,1.80,3723.001515,2108.307692,0.98,4.900000,1.520000,0.188200,5.391715,44.955912,21.0,1.60,1.84,3103.0,57.000000,NaN,3223.000000,2985.000000,1289.00,1814.00,0.294269,85.0,2.15,6000.000000,2108.307692,2.58,111.786885,1.641000,2.077105,10.360010
3,mp-1002124,-19.780759,-9.890380,32.055765,-0.298402,2,HfC,2,9.868236,0.5774,"(Hf, C)",95.250350,39.000000,1.125000,1.375000,4588.000000,71.500000,NaN,1285.517647,7788.500000,1435.000000,3153.000000,0.332135,81.500000,1.965000,10680.000000,1934.153846,1.925000,94.893443,1.601295,0.720092,9.042679,12.0107,6.0,0.70,0.67,4300.00,33.000000,NaN,1285.517647,2267.000000,500.00,2506.00,0.294269,23.00000,1.70,3010.000000,1760.000000,1.30,78.000000,1.580000,0.178070,6.825070,178.490000,72.0,1.55,2.08,4876.0,110.000000,NaN,1285.517647,13310.000000,2370.00,3800.0

In [72]:
# Making list of possible noble gases 
IDs = list(filt_data["material_id"])
IDs = pd.DataFrame(IDs)

train_filt = train[train.index.isin(IDs.index)] 
y_jojo = train_filt.drop('material_id', axis=1)
y_jojo = y_jojo.rename(index=IDs)
#y_jojo


# targets = df.loc[:,["energy",
#         "energy_per_atom",
#         "volume",
#         "formation_energy_per_atom",
#         "nsites",
#         "nelements",
#         "density",
#         "band_gap"]]

# # x_jojo = df[targets]
# # x_jojo

# cols = ["energy",
#         "energy_per_atom",
#         "volume",
#         "formation_energy_per_atom",
#         "nsites",
#         "nelements",
#         "density",
#         "band_gap"] 
# design_matrix= df[cols]

TypeError: 'DataFrame' object is not callable

In [73]:
type(y_jojo)

pandas.core.frame.DataFrame

In [58]:
X_train, X_test, y_train, y_test = train_test_split(design_matrix, 
                                                    y_jojo,
                                                    test_size=0.1, 
                                                    random_state=42)

In [59]:
# Computing mean and standard devaitaion for train X and normalizing
mean_train_X = X_train.apply(np.mean, axis=0)
std_train_X = X_train.apply(np.std, axis=0)
norm_train_X = (X_train - mean_train_X) / std_train_X

# Computing mean and standard devaitaion for test X and normalizing 
mean_test_X = X_test.apply(np.mean, axis=0)
std_test_X = X_test.apply(np.std, axis=0)
norm_test_X = (X_test - mean_test_X) / std_test_X

TypeError: Could not convert mp-556592mp-1771mp-625367mp-756952mp-557134mp-755309mp-30311mp-998762mp-697168mp-4342mp-541487mp-758703mp-22946mp-756138mp-541037mp-567885mp-761341mp-11653mp-20032mp-555863mp-8686mp-571544mp-558891mvc-6611mp-1024052mp-761486mp-8773mp-504422mp-763609mp-771952mp-23558mp-17123mp-9157mp-1215697mp-558430mp-42583mp-767668mp-5858mp-1104363mp-13995mp-1212441mp-1188827mp-1115mp-29488mp-1029633mp-752930mp-676483mvc-6787mp-626799mp-1191341mp-9144mp-37436mp-4351mp-1210700mp-1156mp-28612mp-2097mp-556640mp-6257mp-569857mp-22855mp-1114429mp-5073mp-677335mp-30925mp-634418mp-1114432mp-635425mp-1582mp-867615mp-696752mp-755558mp-561291mp-10695mp-19097mp-1091394mp-9306mp-553923mp-1193265mp-9655mp-19450mp-1209214mp-16737mp-626785mp-4524mp-1120756mp-19147mp-1078191mp-757203mp-505212mp-652046mp-10509mp-504864mp-4584mp-1639mp-18246mp-765046mp-546111mp-567655mp-763784mp-15220mp-9511mp-23366mp-850274mp-1819mp-18244mp-961711mp-1221602mp-555152mp-754496mp-555123mp-14395mp-505381mp-245

In [ ]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
linear_model.fit(norm_train_X, y_train['dielectric_poly_total'])
#linear_predictions = pd.DataFrame(linear_model.predict(norm_test_X),
                                #  columns = ['dielectric_poly_total'])
linear_predictions = linear_model.predict(X_test)

linear_model.score(design_matrix, y_jojo['dielectric_poly_total'])

#Reallinear_prediction = linear_model.predict(test)

In [ ]:
type(linear_predictions)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error


MAE = mean_absolute_error(y_test['dielectric_poly_total'], 
                          linear_predictions)


RMSE = np.sqrt(mean_squared_error(y_test['dielectric_poly_total'], 
                          linear_predictions))


# from sklearn.model_selection import cross_val_predict, KFold
# from sklearn.metrics import r2_score

# # Cross- Validation
# kfold = KFold(n_splits=5, shuffle=True, random_state=42)
# #yhat_mlr = cross_val_predict(linear_model, x_jojo, targets['dielectric_poly_total'], cv=kfold)
# CV_linMAE = mean_absolute_error(targets['dielectric_poly_total'], yhat_mlr)
# CV_linRMSE = np.sqrt(mean_squared_error(targets['dielectric_poly_total'], yhat_mlr))

# print('Cross Val MAE:', CV_linMAE)
# print('Cross Val RMSE:', CV_linRMSE)
print("The MAE is:", MAE)
print("The RMSE is:", RMSE)

In [ ]:
# dielects=(model.predict(test_data))

# df_submit = pd.DataFrame(list(dielects))
# df_submit.to_csv('dielectric_csv.csv')


In [ ]:
# data2 = mpr.query(criteria={"task_id": {"$in":test["material_id"].to_list()}}, properties=["energy",
#         "energy_per_atom",
#         "volume",
#         "formation_energy_per_atom",
#         "nsites",
#         "unit_cell_formula",
#         "pretty_formula",
#         "is_hubbard",
#         "elements",
#         "nelements",
#         "e_above_hull",
#         "hubbards",
#         "is_compatible",
#         "spacegroup",
#         "task_ids",
#         "band_gap",
#         "density",
#         "icsd_id",
#         "icsd_ids",
#         "total_magnetization",
#         "material_id",
#         "oxide_type",
#         "tags",
#         "elasticity"])
# df = pd.DataFrame(data)
# df2 = pd.DataFrame(data2)

In [ ]:
# desired_factors = [ "energy",
#         "energy_per_atom",
#         "volume",
#         "formation_energy_per_atom",
#         "nsites",
#         "nelements",
#         "density",
#         "band_gap"] 

# Y_data = df2[cols]

# #set my model to DecisionTree
# model = DecisionTreeRegressor()

# #set prediction data to factors that will predict, and set target to SalePrice
# train_data = X_data[desired_factors]
# test_data = Y_data[desired_factors]
# target = train.dielectric_poly_total

# # #fitting model with prediction data and telling it my target
# model.fit(train_data, target)

# dielects=(model.predict(test_data))


# dfhi= pd.read_csv("test.csv")

# df_submit = pd.DataFrame(list(dielects))
# df_submit.to_csv('dielectric.csv')

In [ ]:
#dielects

In [ ]:
#len(dielects)

In [ ]:
#test

In [ ]:
 #shit=pd.DataFrame(dielects)

In [ ]:
#d_list=[test, shit]

In [ ]:
#newsubs=pd.concat(d_list, axis=1)

In [ ]:
#newsubs_update=newsubs.rename(columns)

In [ ]:
#newsubs.to_csv("submission_test.csv")

In [ ]:
# from sklearn import metrics
# from sklearn.model_selection import cross_val_predict, KFold
# from sklearn.metrics import r2_score
# # print('Mean absolute error:', metrics.mean_absolute_error(y_test, linear_predictions))
# # print('Mean squared error:', metrics.mean_squared_error(y_test, linear_predictions))
# # print('root mean squared error:', np.sqrt(metrics.mean_squared_error(y_test, linear_predictions)))

# # CV_linMAE = mean_absolute_error(targets['formation_energy_per_atom'], yhat_mlr)
# # CV_linRMSE = np.sqrt(mean_squared_error(targets['formation_energy_per_atom'], yhat_mlr))
# kfold = KFold(n_splits=5, shuffle=True, random_state=42)
# yhat_mlr = cross_val_predict(linear_model, design_matrix, y_jojo['dielectric_poly_total'], cv=kfold)
# metrics.mean_absolute_error(y_test, yhat_mlr)


In [ ]:
# x_train, x_test, y_train, y_test = train_test_split(x_jojo, y_jojo, test_size = 0.2 )

In [ ]:
# x_train.head()

In [ ]:
# # Computing mean and standard devaitaion for train X and normalizing
# mean_train_X = x_train.apply(np.mean, axis=0)
# std_train_X = x_train.apply(np.std, axis=0)
# norm_train_X = (x_train - mean_train_X) / std_train_X

# # Computing mean and standard devaitaion for test X and normalizing 
# mean_test_X = x_test.apply(np.mean, axis=0)
# std_test_X = x_test.apply(np.std, axis=0)
# norm_test_X = (x_test - mean_test_X) / std_test_X


In [ ]:
# # Importing functions for regression 
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
# from sklearn.discriminant_analysis import (LinearDiscriminantAnalysis as LDA, 
#                                            QuadraticDiscriminantAnalysis as QDA)




# linear_model = LinearRegression()
# linear_model.fit(norm_train_X, y_train['dielectric_poly_total'])
# #linear_predictions = pd.DataFrame(linear_model.predict(norm_test_X),
#                                   #columns = ['dielectric_poly_total'])
# linear_predictions = linear_model.predict(norm_test_X)


# mae = mean_absolute_error(y_test, linear_predictions)

# mae





# # from sklearn.metrics import mean_squared_error, mean_absolute_error


# MAE = mean_absolute_error(y_test['dielectric_poly_total'],linear_predictions['dielectric_poly_total'])

# # RMSE = np.sqrt(mean_squared_error(y_test['dielectric_poly_total'], 
# #                           linear_predictions['dielectric_poly_total']))

# print("The MAE is:", MAE)
# # print("The RMSE is:", RMSE)


In [ ]:
#mymodel= np.poly1d(np.polyfit(x_train, y_train, 4 ))

In [ ]:
#r2 = r2_score(y_train, linear_predictions)
